<center>

# $\textbf{Covid-19 - until 2024-02-04}$

<center>

### $\textbf{Code}$

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, year, sum, col, when
import time

In [2]:
inicio = time.time()

In [3]:
spark = SparkSession.builder.appName('Covid').master("local").enableHiveSupport().getOrCreate()
spark

24/03/14 16:31:30 WARN Utils: Your hostname, Francisco-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.191.2.158 instead (on interface en0)
24/03/14 16:31:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 16:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("../Files/Covid.csv", format="csv", sep=",", inferschema="true", header="true")

In [5]:
# Select the columns you're interested in
df = df.select("location","date","total_cases")

In [6]:
# Rename the columns "location" and "total_cases"
df = df.withColumnRenamed("location","country")
df = df.withColumnRenamed("total_cases","covid_cases")

In [7]:
# Extract year from 'day' column
df = df.withColumn("year", year("date"))
df = df.drop("date")

# Replace null values in 'total_cases' column with 0
df = df.withColumn("covid_cases", when(df["covid_cases"].isNull(), 0).otherwise(df["covid_cases"]))

# Remove non-numeric characters from 'total_cases' column
df = df.withColumn("covid_cases", regexp_replace("covid_cases", "[^0-9]", ""))

# Cast columns to their desired types
df = df.withColumn("country", col("country").cast("string"))
df = df.withColumn("year", col("year").cast("int"))
df = df.withColumn("covid_cases", col("covid_cases").cast("int"))

# Filter out any null values after cleaning
df = df.filter(col("covid_cases").isNotNull())

# Calculate total cases per year
df = df.groupBy("country", "year").agg(sum("covid_cases"))

# Order by country and then by year
df = df.orderBy("country", "year")

# Rename 'sum(covid_cases)' to 'covid_cases'
df = df.withColumnRenamed("sum(covid_cases)","covid_cases")

### $\textbf{Pre-Processing}$

In [8]:
# Rename the column
df = df.withColumnRenamed("Micronesia (country)","Micronesia")
df = df.withColumnRenamed("Saint Martin (French part)","Saint Martin")
df = df.withColumnRenamed("Sint Maarten (Dutch part)","Sint Maarten")
df = df.withColumnRenamed("Brunei","Brunei Darussalam")
df = df.withColumnRenamed("Cape Verde","Cabo Verde")
df = df.withColumnRenamed("Laos","Lao PDR")
df = df.withColumnRenamed("United States Virgin Islands","Virgin Islands (U.S.)")
df = df.withColumnRenamed("Turkey","Turkiye")
df = df.withColumnRenamed("Syria","Syrian Arab Republic")
df = df.withColumnRenamed("Timor","Timor-Leste")
df = df.withColumnRenamed("Russia","Russian Federation")
df = df.withColumnRenamed("Congo","Democratic Republic of the Congo")
df = df.withColumnRenamed("Slovakia","Slovenia")

In [9]:
# Drop the specified lines
countries_to_remove = ["Africa", "Europe", "Europe Union", "Asia", "Lower middle income", "Upper middle income", "World"]
df = df.filter(~df["country"].isin(countries_to_remove))

In [10]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("../FilesParquet/Covid.parquet" )
spark.read.parquet("../FilesParquet/Covid.parquet").show()
spark.stop()

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=23582Kb max_used=23595Kb free=107489Kb
 bounds [0x000000010a9e0000, 0x000000010c110000, 0x00000001129e0000]
 total_blobs=9256 nmethods=8291 adapters=878
 compilation: disabled (not enough contiguous free space left)


+--------------+----+-----------+
|       country|year|covid_cases|
+--------------+----+-----------+
|   Afghanistan|2020|   82849920|
|   Afghanistan|2021|  391774200|
|   Afghanistan|2022|  675763230|
|   Afghanistan|2023|  802611370|
|   Afghanistan|2024|   80812150|
|       Albania|2020|   34448550|
|       Albania|2021|  503635430|
|       Albania|2022| 1072461670|
|       Albania|2023| 1218905130|
|       Albania|2024|  117160410|
|       Algeria|2020|  101885690|
|       Algeria|2021|  567017170|
|       Algeria|2022|  963218050|
|       Algeria|2023|  991880860|
|       Algeria|2024|   95203500|
|American Samoa|2020|          0|
|American Samoa|2021|       5420|
|American Samoa|2022|   20366540|
|American Samoa|2023|   30429780|
|American Samoa|2024|    2925650|
+--------------+----+-----------+
only showing top 20 rows



In [11]:
fim = time.time()
final = fim - inicio
print(final)

9.594932794570923
